# Grabing the montly data on drug seizures published by the US Customs and Border Patrol

The US CBP update a [dashboard](https://www.cbp.gov/newsroom/stats/drug-seizure-statistics) monthly and also provide CSVs of the data [here](https://www.cbp.gov/document/stats/nationwide-drug-seizures). The scaper in this script is going to go through the following steps:
1. grab the HTML of the website with the CSV files
2. grab the date of the latest CSV
3. compare the date from step 2 to the date of the most recent dataset we have
4. if the date from step 2 equals the date of our most recent data, the script stops, if it is more recent, we download the lastest CSV

Before actually writing our code, we need to import the libraries that we're going to use.

In [5]:
import requests
import urllib3
from bs4 import BeautifulSoup
from datetime import datetime
import re
import pandas as pd
import os

## 1. grab the HTML of the website with the CSV files

In [7]:
# Specify the URL
url = "http://www.cbp.gov/document/stats/nationwide-drug-seizures"

# Make the website think we're a normal person browsing, not a bot
urllib3.disable_warnings()

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
}

# Get the HTML from the URL
response = requests.get(url, headers=headers)

This next cell is options and will be deleted before the code goes into production. If the last step is successful, the computer returns the status 200. So it we get that status, the notebook prints the code so we can see it, and it not, it tells us we have an error.

In [9]:
if response.status_code == 200:
    #print(response.text)  
    print('Page downloaded sucessfully')
else:
    print(f'Request failed with status code: {response.status_code}')

Page downloaded sucessfully


## 2. grab the date of the latest CSV

This code is going to take the HTML we just grabbed, as select the first row of a table on the page. The way the HTML is structred[1], 

[1] As of my writing of this code. Websites are often updated, and if the CBP change their website, this code will have to be modified accordingly.

In [12]:
# Parse the content with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find all <td> tags
table_tags = soup.find_all('td')[2]

# Strip out the HTML
latest_date_str = re.sub(r'\s+', '', re.sub('</td>', '', re.sub('<td.*>', '', str(table_tags))))

latest_date = datetime.strptime(latest_date_str, "%m/%d/%Y").date()
print(latest_date)

2024-04-08


## 3. if the date from step 2 equals the date of our most recent data, the script stops, if it is more recent, we download the lastest CSV

In step 4, we compare two dates and then best on the results, call different functions. In order for that to work, we need to have the functions already defined. So here, we'll define all the options, then we'll run the comparison and the program will execute whichever function is needed.

In [15]:
base_url = "https://www.cbp.gov"
def get_dataset():
    link_tags = soup.find('tbody')
    link_stub = link_tags.find('a').get('href')

    full_url = base_url + link_stub
    filename = "data/cbp_data_" + str(latest_date) + ".csv"

    query_parameters = {"downloadformat": "csv"}
    data_response = requests.get(full_url, params=query_parameters)
    if data_response.status_code == 200:
        with open(filename, mode="wb") as file:
            file.write(data_response.content)
    else:
        raise ValueError(f'Request failed with status code: {data_response.status_code}')

## 4. compare the date from step 2 to the date of the most recent dataset we have

This next cell looks at all the files saved in the `data` folder. It strips the date from the file names and assigns the most recent date to the `current_date` variable.

In [18]:
if len(os.listdir('data/')) > 0:
    matches = [file for file in os.listdir('data/') if "cbp_data_" in file]
    date_str = ([datetime.strptime(match.replace('cbp_data_', '')
                                   .replace('.csv', ''), "%Y-%m-%d")
                 .date() for match in matches])
    current_date = max(date_str)
else:
    raise ValueError('Error:\nNo files in the "data/" directory')

The function `compare_dates` is going to take the variable `current_date` and compare it to the variable `latest_date` we created in step 2. When `compare_dates` is earlier than `current_date`, it will call the `get_dataset()` function to we defined in step 3. If `current_date` is equal to the `latest_date`, that means we already have the latest dataset and the program stops. If the `current_date` is greater than the `latest_date`, that means there's and error or the CBP has taken down some of their data. This needs to be investigated.

In [20]:
def compare_dates(latest_date):
    if current_date < latest_date:
        get_dataset()
    elif current_date == latest_date:
        print("Dataset is already up to date")
    else:
        raise ValueError("Error:\nSomething is wrong. The current dataset\nseems to be more recent than the most\nrecent data. You should investigate whether\nthe site structure has changed or a dataset\nwas removed.")
        
compare_dates(latest_date)

Dataset is already up to date
